## 1. Counting missing values in the tables

In [195]:
%%sql
postgresql:///sports
-- Count all columns from the info table, aliasing as total_rows
-- Count the number of non-missing entries for description, listing_price, and last_visited
-- Join info, finance, and traffic

SELECT COUNT(*) as total_rows, COUNT(i.description) as count_description, COUNT(f.listing_price) as count_listing_price, COUNT(t.last_visited) as count_last_visited
from info i
Join finance f on i.product_id = f.product_id
Join traffic t on i.product_id = t.product_id


1 rows affected.


total_rows,count_description,count_listing_price,count_last_visited
3179,3117,3120,2928


## 2. Nike vs Adidas pricing.

In [197]:
%%sql

-- Select the brand, listing_price as an integer, and a count of all products in finance 
-- Join brands to finance on product_id
-- Filter for products with a listing_price more than zero
-- Aggregate results by brand and listing_price, and sort the results by listing_price in descending order

SELECT b.brand, CAST(f.listing_price AS INTEGER), COUNT(f.*)
from finance f JOIN brands b
ON f.product_id = b.product_id
WHERE f.listing_price > 0
GROUP BY b.brand, f.listing_price
ORDER BY f.listing_price desc

 * postgresql:///sports
77 rows affected.


brand,listing_price,count
Adidas,300,2
Adidas,280,4
Adidas,240,5
Adidas,230,8
Adidas,220,11
Adidas,200,8
Nike,200,1
Adidas,190,7
Nike,190,2
Adidas,180,34


## 3. Labeling price ranges for each of the brands in categories "Budget", "Average", "Expensive", "Elite".

In [199]:
%%sql

-- Select the brand, a count of all products in the finance table, and total revenue
-- Create four labels for products based on their price range, aliasing as price_category
-- Join brands to finance on product_id and filter out products missing a value for brand
-- Group results by brand and price_category, sort by total_revenue

SELECT b.brand, COUNT(f.*), SUM(f.revenue) as total_revenue, 
    CASE 
        WHEN f.listing_price < 42 THEN 'Budget'
        WHEN f.listing_price >= 42 AND f.listing_price < 74  THEN 'Average'
        WHEN f.listing_price >= 74 AND f.listing_price < 129  THEN 'Expensive'
        WHEN f.listing_price > 129  THEN 'Elite'
    END as price_category
from finance f JOIN brands b 
on f.product_id = b.product_id
WHERE b.brand is not NULL
GROUP BY b.brand, price_category
ORDER BY total_revenue desc

 * postgresql:///sports
8 rows affected.


brand,count,total_revenue,price_category
Adidas,849,4626980.069999999,Expensive
Adidas,1060,3233661.060000001,Average
Adidas,307,3014316.8299999987,Elite
Adidas,359,651661.1200000002,Budget
Nike,357,595341.0199999992,Budget
Nike,82,128475.59000000003,Elite
Nike,90,71843.15000000004,Expensive
Nike,16,6623.5,Average


## 4. Average discount by brand

In [201]:
%%sql

-- Select brand and average_discount as a percentage
-- Join brands to finance on product_id
-- Aggregate by brand
-- Filter for products without missing values for brand

SELECT b.brand, AVG(f.discount) * 100 as average_discount
from finance f JOIN brands b 
on f.product_id = b.product_id
WHERE b.brand is not NULL
GROUP BY b.brand

 * postgresql:///sports
2 rows affected.


brand,average_discount
Nike,0.0
Adidas,33.452427184465606


## 5. Correlation between revenue and reviews

In [203]:
%%sql

-- Calculate the correlation between reviews and revenue as review_revenue_corr
-- Join the reviews and finance tables on product_id

SELECT CORR(r.reviews, f.revenue) as review_revenue_corr
FROM reviews r JOIN finance f
ON r.product_id = f.product_id

 * postgresql:///sports
1 rows affected.


review_revenue_corr
0.6518512283481301


## 6. Ratings and reviews by product description length

In [205]:
%%sql

-- Calculate description_length
-- Convert rating to a numeric data type and calculate average_rating
-- Join info to reviews on product_id and group the results by description_length
-- Filter for products without missing values for description, and sort results by description_length

SELECT TRUNC(LENGTH(description) / 100) * 100 as description_length, ROUND(AVG(r.rating::numeric), 2) as average_rating
from info i JOIN reviews r
On i.product_id = r.product_id
WHERE i.description is not NULL
Group By description_length
ORDER BY description_length

 * postgresql:///sports
7 rows affected.


description_length,average_rating
0.0,1.87
100.0,3.21
200.0,3.27
300.0,3.29
400.0,3.32
500.0,3.12
600.0,3.65


## 7. Reviews by month and brand

In [207]:
%%sql

-- Select brand, month from last_visited, and a count of all products in reviews aliased as num_reviews
-- Join traffic with reviews and brands on product_id
-- Group by brand and month, filtering out missing values for brand and month
-- Order the results by brand and month

SELECT b.brand, EXTRACT(MONTH from t.last_visited) as month, COUNT(r.*) as num_reviews
FROM traffic t 
JOIN reviews r on t.product_id = r.product_id
JOIN brands b on t.product_id = b.product_id
WHERE b.brand is not NULL AND EXTRACT(MONTH from t.last_visited) is not NULL
GROUP BY b.brand, month
ORDER BY b.brand, month

 * postgresql:///sports
24 rows affected.


brand,month,num_reviews
Adidas,1,253
Adidas,2,272
Adidas,3,269
Adidas,4,180
Adidas,5,172
Adidas,6,159
Adidas,7,170
Adidas,8,189
Adidas,9,181
Adidas,10,192


## 8. Footwear product performance

In [209]:
%%sql

-- Create the footwear CTE, containing description and revenue
-- Filter footwear for products with a description containing %shoe%, %trainer, or %foot%
-- Also filter for products that are not missing values for description
-- Calculate the number of products and median revenue for footwear products

WITH footwear as (
    SELECT i.description, f.revenue from info i
    Join finance f on i.product_id = f.product_id
    WHERE i.description is not NULL And (i.description ILIKE '%shoe%' OR i.description ILIKE '%trainer%' OR i.description ILIKE '%foot%')
)
SELECT COUNT(*) as num_footwear_products, percentile_disc(0.5) WITHIN GROUP (ORDER BY revenue) as median_footwear_revenue
from footwear

 * postgresql:///sports
1 rows affected.


num_footwear_products,median_footwear_revenue
2700,3118.36


## 9. Clothing product performance

In [211]:
%%sql

-- Copy the footwear CTE from the previous task
-- Calculate the number of products in info and median revenue from finance
-- Inner join info with finance on product_id
-- Filter the selection for products with a description not in footwear
WITH footwear as (
    SELECT i.description, f.revenue from info i
    Join finance f on i.product_id = f.product_id
    WHERE i.description is not NULL And (i.description ILIKE '%shoe%' OR i.description ILIKE '%trainer%' OR i.description ILIKE '%foot%')
)
SELECT COUNT(i.*) as num_clothing_products, percentile_disc(0.5) WITHIN GROUP (ORDER BY f.revenue) as median_clothing_revenue
FROM info i
Join finance f on i.product_id = f.product_id
WHERE i.description is not NULL AND i.description not in (SELECT description from footwear )

 * postgresql:///sports
1 rows affected.


num_clothing_products,median_clothing_revenue
417,503.82
